In [134]:
import json_lines as jl
import nltk
import xmltodict
from ispras import texterra as isp

In [129]:
comments = ''
with open('learn_scrapy\pda.jl', 'rb') as f:
    for item in jl.reader(f):
        comments = comments + item['comment']

In [105]:
tknzr = nltk.TweetTokenizer()
tokens = tknzr.tokenize(comments)
wrds = [w.lower() for w in tokens if w.isalpha()]
vocab = sorted(set(wrds))
text = nltk.Text(wrds)

In [106]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

print(lexical_diversity(text))

0.11315910636838525


In [120]:
stemmer = nltk.SnowballStemmer("russian")
stemmed = [stemmer.stem(t) for t in wrds]
print(wrds[0:100])
print(stemmed[0:100])

['если', 'она', 'только', 'на', 'алроиды', 'и', 'айос', 'вышла', 'где', 'была', 'раньше', 'в', 'помойке', 'не', 'знаю', 'зачем', 'ее', 'от', 'туда', 'достали', 'рескин', 'что', 'за', 'наркоманский', 'сультфильм', 'потом', 'еще', 'говорят', 'что', 'аниме', 'плохо', 'хоспаде', 'опять', 'старперы', 'повылезали', 'сидите', 'там', 'свои', 'нупогоди', 'смотрите', 'я', 'могу', 'ошибаться', 'но', 'как', 'раз', 'старперы', 'и', 'помнят', 'футураму', 'раз', 'смотрел', 'все', 'сезоны', 'есть', 'коллекция', 'на', 'блюрей', 'глупый', 'но', 'жутко', 'угарный', 'мультик', 'а', 'игра', 'ггг', 'лучше', 'бы', 'платформер', 'какой', 'запилили', 'поддерживаю', 'классный', 'угарный', 'мульт', 'с', 'шутками', 'которые', 'нужно', 'слушать', 'а', 'не', 'смотреть', 'ну', 'и', 'понимать', 'разумеется', 'good', 'news', 'everyone', 'много', 'уважаемый', 'за', 'чем', 'изначально', 'коверкать', 'слово', 'господи', 'или', 'вы']
['есл', 'он', 'тольк', 'на', 'алроид', 'и', 'айос', 'вышл', 'где', 'был', 'раньш', 'в', '

In [238]:
mini_text = comments[0:100000]
t = texterra.API('dd58e71c2c65fa4af14fb557f904ede7f3b30575')
a = t.lemmatizationAnnotate(mini_text)

In [ ]:
lst = a.get('annotations').get('lemma')
new_text = ''
#for sample in lst:
    #print(sample.get('text'))
    #print(sample.get('value'))

#print(new_text)
#print(mini_text)

еслионатольконаалроидиайосвыходитьгдебытьраноо_овпомойканезнатьзачемонаоттудадоставатьрескинчтозанаркоманскийсультфильмпотомещеговоритьчтоанимплохохоспадопятьстарперповылезатьсидетьтамсвойнупогодьсмотретьямочьошибатьсянопо-моемукакразстарперипомнитьфутурамараз10смотретьвесьсезонбытьколлекциянаблюрейглупыйножуткоугарныймультикаиграгггодхорошобыплатформиратькакойзапилилиподдерживаю,классныйугарныймультсшуткикоторыйнужныйслушатьанесмотретьнуипониматьразумеетсяgoodnewseveryoneмногоуважатьзачтоизначальноковеркатьсловогосподьиливысчитатьчтотаквыумныйбытьвыглядетьизачтовообщеупотребльвданныйконтекстэтослововыбыхорошонена4pdaсидетьанакакойнятьпознавательныйсайтзаодинсвойlvlпрокачинатьбыавыстатьвыглядетьболееумныйпослеэтотвашкомментарияхоспадчастоиспользоватьвмассакультураиужеустойчивыйвыражениеионэтотничтонехотетьпоказыватьилиещечтопростореакциянакритикабезаргументэтожекакойстарыйнадобытьчтобынезнатьчтозамультфильмнезнатьизнатьнехотетьэтотамериканскийнаркоттамгдеговоретьголовавбанкбытьвмодатот

In [165]:
len(comments)

3406157